По традиции начинаем с вопроса....

У нас есть множество различных новостных статей на абсолютно разные темы. Нам нужно определить тему статьи (узкую) (найти в массе из 3000+ знаков какие-то самые значимые для этой статьи слова, которые делают её отличной от других статей и которые могут поступить в заголовок и составлять её  краткий пересказ) --- как нам это сделать? 


Очевидным образом, не все слова were created equal. Мы уже знакомы с рядос стоп слов, которые мы в лингвистике яростно выбрасываем из предложения, однако, и другие слова в тексте неравнознчны.......

# TF_IDF

Что это? И зачем оно нужно? 

По сути, это метрика, которая показывает насколько важно/релевантно слово в заданном документе. используется для анализа темы и в поисковиках.

В этом семинаре мы попытаемся создать с помощью неё наш собственный поисковик.

Представим, что нам на вход в поисковике поступило 'the brown cows'. У нас в мировой паутине лежит куча-куча-куча документов с множеством различных слов. Как по заданным словам понять, какой из них релевантнее? 

Пока забьём на морфологию для простоты 

In [1]:
import pandas as pd # если нет pandas - не страшно - он тут для красоты


In [2]:
inquery = 'the brown cows'

documentA = 'the black man went out at night for a stroll past the brown cows he owned He also noticed a wolf on the stroll but did not pay attention ...'
documentB = 'The brown children sat around the fire and sang the songs ...'
documentC = 'Everything about brown cows.  Brown cows live in a harsh enviroment. The ... '
documentD = 'Everything about cows. Cows live in different enviroments. The ...'

Как мы можем, не залезая в семантику, узнать, какой текст выдать первым?

tf


idf


term frequency–inverse document frequency

In [3]:

import nltk
nltk.download('punkt')
bagOfWordsA = nltk.word_tokenize(documentA.lower())
bagOfWordsB = nltk.word_tokenize(documentB.lower())
bagOfWordsC = nltk.word_tokenize(documentC.lower())
bagOfWordsD = nltk.word_tokenize(documentD.lower())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avorl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB)).union(set(bagOfWordsC)).union(set(bagOfWordsD))

In [5]:
uniqueWords

{'.',
 '...',
 'a',
 'about',
 'also',
 'and',
 'around',
 'at',
 'attention',
 'black',
 'brown',
 'but',
 'children',
 'cows',
 'did',
 'different',
 'enviroment',
 'enviroments',
 'everything',
 'fire',
 'for',
 'harsh',
 'he',
 'in',
 'live',
 'man',
 'night',
 'not',
 'noticed',
 'on',
 'out',
 'owned',
 'past',
 'pay',
 'sang',
 'sat',
 'songs',
 'stroll',
 'the',
 'went',
 'wolf'}

In [6]:
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1

numOfWordsB = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsB:
    numOfWordsB[word] += 1
    
numOfWordsC = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsC:
    numOfWordsC[word] += 1
    
numOfWordsD = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsD:
    numOfWordsD[word] += 1
    
df = pd.DataFrame([numOfWordsA, numOfWordsB,numOfWordsC, numOfWordsD ])    

In [7]:
pd.set_option('display.max_columns', None)
df

,owned,harsh,and,pay,everything,at,cows,he,also,out,around,songs,about,the,children,sat,black,noticed,stroll,did,went,...,attention,night,but,live,a,enviroments,past,.,for,fire,not,in,brown,man,on,enviroment,different,wolf,sang
0,1,0,0,1,0,1,1,2,1,1,0,0,0,3,0,0,1,1,2,1,1,1,1,1,1,0,2,0,1,0,1,0,1,0,1,1,1,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,1,1,0,3,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
2,0,1,0,0,1,0,2,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,2,0,0,0,1,2,0,0,1,0,0,0
3,0,0,0,0,1,0,2,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,2,0,0,0,1,0,0,0,0,1,0,0


Почему иногда всё же но стоит выкидывать стоп-слова - предлоги, артикли и союзы?


## давайте построим свой tf-idf

## TF

Tf --- term frequency. В самом первом упрощении - это то, сколько раз слово встретилось в тексте (чем чаще искомое слово встретилось в заданном тексте по сравнению с другими, тем это слово релевантнее для текста (лучше отражает его тему) и рем этот текст релевантнее запросу), в чём тут проблема? 


Поэтому есть всякие adjustments. Так часто tf слова в документе считается как количество раз, когда оно появилось в этом документе, деленное на общее число слов документе.

In [8]:
def computeTF(wordDict, bagOfWords):
    #напишите функцию, которая получает на ввход bagofwords и numOfWords, а возвращает словарь с tf-ами для каждого слова из numofwords
    
    return tfDict

In [9]:
tfA = computeTF(numOfWordsA, bagOfWordsA)
tfB = computeTF(numOfWordsB, bagOfWordsB)
tfC = computeTF(numOfWordsC, bagOfWordsC)
tfD = computeTF(numOfWordsD, bagOfWordsD)

In [10]:
tfB

{'owned': 0.0,
 'harsh': 0.0,
 'and': 0.08333333333333333,
 'pay': 0.0,
 'everything': 0.0,
 'at': 0.0,
 'cows': 0.0,
 'he': 0.0,
 'also': 0.0,
 'out': 0.0,
 'around': 0.08333333333333333,
 'songs': 0.08333333333333333,
 'about': 0.0,
 'the': 0.25,
 'children': 0.08333333333333333,
 'sat': 0.08333333333333333,
 'black': 0.0,
 'noticed': 0.0,
 'stroll': 0.0,
 'did': 0.0,
 'went': 0.0,
 '...': 0.08333333333333333,
 'attention': 0.0,
 'night': 0.0,
 'but': 0.0,
 'live': 0.0,
 'a': 0.0,
 'enviroments': 0.0,
 'past': 0.0,
 '.': 0.0,
 'for': 0.0,
 'fire': 0.08333333333333333,
 'not': 0.0,
 'in': 0.0,
 'brown': 0.08333333333333333,
 'man': 0.0,
 'on': 0.0,
 'enviroment': 0.0,
 'different': 0.0,
 'wolf': 0.0,
 'sang': 0.08333333333333333}

# idf

idf --- The log of the number of documents divided by the number of documents that contain the word w. Inverse data frequency determines the weight of rare words across all documents in the corpus.

In [11]:
import math
def computeIDF(documents):
    #для того, чтобы посчитать idf надо подать все документы в функцию сразу 
    #напишите функцию, считающую idf для каждого слова - на входе массив из numOfWords для всех текстов 
    #на выходе словарь для слов 
    #см. ввод и вывод ниже
    
    return idfDict

In [12]:
idfs = computeIDF([numOfWordsA, numOfWordsB, numOfWordsC, numOfWordsD])

In [13]:
idfs

{'owned': 1.3862943611198906,
 'harsh': 1.3862943611198906,
 'and': 1.3862943611198906,
 'pay': 1.3862943611198906,
 'everything': 0.6931471805599453,
 'at': 1.3862943611198906,
 'cows': 0.28768207245178085,
 'he': 1.3862943611198906,
 'also': 1.3862943611198906,
 'out': 1.3862943611198906,
 'around': 1.3862943611198906,
 'songs': 1.3862943611198906,
 'about': 0.6931471805599453,
 'the': 0.0,
 'children': 1.3862943611198906,
 'sat': 1.3862943611198906,
 'black': 1.3862943611198906,
 'noticed': 1.3862943611198906,
 'stroll': 1.3862943611198906,
 'did': 1.3862943611198906,
 'went': 1.3862943611198906,
 '...': 0.0,
 'attention': 1.3862943611198906,
 'night': 1.3862943611198906,
 'but': 1.3862943611198906,
 'live': 0.6931471805599453,
 'a': 0.6931471805599453,
 'enviroments': 1.3862943611198906,
 'past': 1.3862943611198906,
 '.': 0.6931471805599453,
 'for': 1.3862943611198906,
 'fire': 1.3862943611198906,
 'not': 1.3862943611198906,
 'in': 0.6931471805599453,
 'brown': 0.28768207245178085,

## tf_idf  целиком

тф_айдф для слова в документе - это просто произведение его тф в документе на его идф

In [14]:
def computeTFIDF(tfBagOfWords, idfs):
    #давайте
    
    return tfidf

In [15]:
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
tfidfC = computeTFIDF(tfC, idfs)
tfidfD = computeTFIDF(tfD, idfs)
df = pd.DataFrame([tfidfA, tfidfB, tfidfC, tfidfD])

In [16]:
df

,owned,harsh,and,pay,everything,at,cows,he,also,out,around,songs,about,the,children,sat,black,noticed,stroll,did,went,...,attention,night,but,live,a,enviroments,past,.,for,fire,not,in,brown,man,on,enviroment,different,wolf,sang
0,0.04621,0.00000,0.000000,0.04621,0.000000,0.04621,0.009589,0.09242,0.04621,0.04621,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.04621,0.04621,0.09242,0.04621,0.04621,0.0,0.04621,0.04621,0.04621,0.000000,0.04621,0.000000,0.04621,0.000000,0.04621,0.000000,0.04621,0.000000,0.009589,0.04621,0.04621,0.00000,0.000000,0.04621,0.000000
1,0.00000,0.00000,0.115525,0.00000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.115525,0.115525,0.000000,0.0,0.115525,0.115525,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.115525,0.00000,0.000000,0.023974,0.00000,0.00000,0.00000,0.000000,0.00000,0.115525
2,0.00000,0.09242,0.000000,0.00000,0.046210,0.00000,0.038358,0.00000,0.00000,0.00000,0.000000,0.000000,0.046210,0.0,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.046210,0.04621,0.000000,0.00000,0.092420,0.00000,0.000000,0.00000,0.046210,0.038358,0.00000,0.00000,0.09242,0.000000,0.00000,0.000000
3,0.00000,0.00000,0.000000,0.00000,0.057762,0.00000,0.047947,0.00000,0.00000,0.00000,0.000000,0.000000,0.057762,0.0,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.057762,0.00000,0.115525,0.00000,0.115525,0.00000,0.000000,0.00000,0.057762,0.000000,0.00000,0.00000,0.00000,0.115525,0.00000,0.000000


посмотрим на brown

## Автоматизация

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB, documentC, documentD])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [19]:
df

,about,also,and,around,at,attention,black,brown,but,children,cows,did,different,enviroment,enviroments,everything,fire,for,harsh,he,in,live,man,night,not,noticed,on,out,owned,past,pay,sang,sat,songs,stroll,the,went,wolf
0,0.000000,0.184851,0.000000,0.000000,0.184851,0.184851,0.184851,0.117988,0.184851,0.000000,0.117988,0.184851,0.000000,0.000000,0.000000,0.000000,0.000000,0.184851,0.000000,0.369701,0.000000,0.000000,0.184851,0.184851,0.184851,0.184851,0.184851,0.184851,0.184851,0.184851,0.184851,0.000000,0.000000,0.000000,0.369701,0.289388,0.184851,0.184851
1,0.000000,0.000000,0.318493,0.318493,0.000000,0.000000,0.000000,0.203290,0.000000,0.318493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318493,0.318493,0.318493,0.000000,0.498608,0.000000,0.000000
2,0.278433,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.450831,0.000000,0.000000,0.450831,0.000000,0.000000,0.353157,0.000000,0.278433,0.000000,0.000000,0.353157,0.000000,0.278433,0.278433,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.184292,0.000000,0.000000
3,0.311932,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.505071,0.000000,0.395646,0.000000,0.395646,0.311932,0.000000,0.000000,0.000000,0.000000,0.311932,0.311932,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.206464,0.000000,0.000000


Теперь даваайте создадим наш парсер

In [20]:
import csv
import pymorphy2
import string
import re
from math import log
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

morph = pymorphy2.MorphAnalyzer()
vectorizer1 = CountVectorizer(binary = True)
vectorizertf = TfidfVectorizer(use_idf=False)

quora

['', 'question1', 'question2', 'is_duplicate']

['0', 'Какова история кохинор кох-и-ноор-бриллиант', 'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад', '0']

['1', 'как я могу увеличить скорость моего интернет-соединения, используя vpn', 'как повысить скорость интернета путем взлома через dns', '0']

['2', 'почему я мысленно очень одинок, как я могу это решить', 'найти остаток, когда математика 23 ^ 24 математика разделена на 24 23', '0']

['3', 'которые растворяют в воде быстро сахарную соль метан и углеродный диоксид', 'какая рыба выживет в соленой воде', '0']

['4', 'астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне', 'Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне', '1']

давайте обработаем только  20 000

In [21]:
def get_texts(): # возвращаем [corpus, inquiery, scores, normal]
    corpus = [] #сюда кладём тексты из [2] столбца, где каждое слово прошло препроцессинг
                #(нижний регистр, нет знаков препинания) и каждое слово в начальной форме
    inquiery = [] #сюда кладём тексты из [1] столбца - их препроцесить не надо
    scores = [] #сюда кладём информацию из [3] -- является ли ответ соответсвующим вопросу
    normal = [] #сюда кладём тексты из [2] столбца - непрепроцесенные

    with open('quora_question_pairs_rus.csv', encoding = 'utf-8') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            # отсюда убрать
           

In [22]:
data = get_texts()

In [23]:
corpus_texts, inquiery_texts, scores, normal_texts =  data[0][1:], data[1][1:], data[2][1:], data[3][1:]
del data

In [24]:
print(corpus_texts[:5])
print(inquiery_texts[:5])
print(scores[:5])

['что произойти если правительство индия украсть кохинор кох-и-ноор-алмаз назад', 'как повысить скорость интернет путём взлом через dns', 'найти остаток когда математик ^ математик разделить на', 'какой рыба выжить в солёный вода', 'я тройной луна-козерог и восхождение в козерог что это говорить о я']
['Какова история кохинор кох-и-ноор-бриллиант', 'как я могу увеличить скорость моего интернет-соединения, используя vpn', 'почему я мысленно очень одинок, как я могу это решить', 'которые растворяют в воде быстро сахарную соль метан и углеродный диоксид', 'астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне']
['0', '0', '0', '0', '1']


In [25]:
corpus_matrix1 = vectorizer1.fit_transform(corpus_texts)
voc = vectorizer1.get_feature_names() #получили вокабуляр -- какие вообще есть слова, у каждого еще и свой индекс

In [28]:
voc [70:100]

['airtel',
 'aiyoo',
 'ajax',
 'ajay',
 'aka',
 'akamai',
 'akc',
 'al',
 'alabama',
 'alan',
 'alanganallur',
 'albany',
 'alchemyapi',
 'alchievers',
 'alephti',
 'aleppo',
 'alex',
 'alexa',
 'ali',
 'alia',
 'alibaba',
 'aliexpress',
 'allahabad',
 'allbestlist',
 'allen',
 'allo',
 'almos',
 'alpha',
 'als',
 'am']

вообще для этих целей лучше использовать словарь, но...

In [29]:
tf = vectorizertf.fit_transform(corpus_texts)
tf = tf.toarray() # это матрица!!!

In [30]:
def search_mat( inquiery, voc):
    #делаем препроцессинг инквери
    inq = preproc(inquiery)
    # создали вектор из нулей, равный нашему вакабуляру по длине
    vec = np.zeros((len(voc)))
    for word in inq.split(' '):
        if word in voc:
            index = voc.index(word)
            vec[index] = 1 # нули начинают заполняться единицами
    res = np.dot(tf,vec) #перемножили вектора и матрицы --- получили что? -- 
    # https://zaochnik.com/spravochnik/matematika/matritsy/umnozhenie-matrits/
    #по сути, те ребята, у которых не было ни одного слова совпадающего с запросом сразу редуцировались в ноль
    # а у тех, у кого было мы с помощью такого умножения определили им , ктот релевантнее через тф_идф, получив числа
    
    results = np.argsort(res)[::-1][:5]
    
   
    for i in results:
        print(normal_texts[i])
    return

In [31]:
search_mat('правительство индии',voc)


как работает правительство Индии
почему правительство Индии запрещает все
какая разница между парламентом и правительством Индии
есть ли квота в Индии в Индии?
каковы преимущества решения правительства Индии о вывозе 500 и 1000 рупий


Много переменных +,если это не quora, персонализация